In [27]:
import torch
import cv2
import sys
import os
from torchvision import transforms
sys.path.append(os.path.abspath('../src'))
from models.cnn_model import Custom_CNN

class StateClassification:
    def __init__(self, eye_model_path, mouth_model_path):
        # Initialize models with the same parameters used during training
        self.eye_model = Custom_CNN(
            input_shape=1,  # Grayscale input
            hidden_units=10,  # Adjust based on your training parameters
            output_shape=2   # Binary classification (open/closed)
        )
        self.mouth_model = Custom_CNN(
            input_shape=1,
            hidden_units=10,
            output_shape=2   # Binary classification (yawning/not yawning)
        )
        
        # Load state dictionaries
        self.eye_model.load_state_dict(torch.load(eye_model_path, weights_only=True))
        self.mouth_model.load_state_dict(torch.load(mouth_model_path,weights_only=True))
        
        # Set models to evaluation mode
        self.eye_model.eval()
        self.mouth_model.eval()
        
        # Define image transformations
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

    def classify_eye(self, eye_image):
        # Convert image to grayscale if it's RGB
        if len(eye_image.shape) == 3:
            eye_image = cv2.cvtColor(eye_image, cv2.COLOR_RGB2GRAY)
            
        # Resize image to match model's expected input size (224x224 for example)
        eye_image = cv2.resize(eye_image, (224, 224))  # Adjust size according to your model's training
        
        with torch.no_grad():  # Disable gradient calculation
            # Add channel dimension for grayscale image
            image_tensor = self.transform(eye_image).unsqueeze(0)  # Add batch dimension
            output = self.eye_model(image_tensor)
            return torch.argmax(output).item()  # 0: Open, 1: Closed

    def classify_mouth(self, mouth_image):
        # Convert image to grayscale if it's RGB
        if len(mouth_image.shape) == 3:
            mouth_image = cv2.cvtColor(mouth_image, cv2.COLOR_RGB2GRAY)
            
        # Resize image to match model's expected input size
        mouth_image = cv2.resize(mouth_image, (224, 224))  # Adjust size according to your model's training
        
        with torch.no_grad():  # Disable gradient calculation
            # Add channel dimension for grayscale image
            image_tensor = self.transform(mouth_image).unsqueeze(0)  # Add batch dimension
            output = self.mouth_model(image_tensor)
            return torch.argmax(output).item()  # 0: Not yawning, 1: Yawning

In [31]:
state_classifier = StateClassification("/Users/ahmedalkhulayfi/Desktop/Real-Time-Driver-Drowsiness-Detection/notebooks/best_model.pth", "/Users/ahmedalkhulayfi/Desktop/Real-Time-Driver-Drowsiness-Detection/notebooks/best_model.pth")

In [15]:
# Add this code temporarily to debug the model structure
eye_model_path = "/Users/ahmedalkhulayfi/Desktop/Real-Time-Driver-Drowsiness-Detection/src/models/eye_state_model.pth"
checkpoint = torch.load(eye_model_path, map_location='cpu')
print("Checkpoint keys:", checkpoint.keys())
if 'state_dict' in checkpoint:
    print("\nState dict keys:", checkpoint['state_dict'].keys())
else:
    print("\nState dict keys:", checkpoint.keys())

Checkpoint keys: odict_keys(['features.0.0.weight', 'features.0.1.weight', 'features.0.1.bias', 'features.0.1.running_mean', 'features.0.1.running_var', 'features.0.1.num_batches_tracked', 'features.1.0.block.0.0.weight', 'features.1.0.block.0.1.weight', 'features.1.0.block.0.1.bias', 'features.1.0.block.0.1.running_mean', 'features.1.0.block.0.1.running_var', 'features.1.0.block.0.1.num_batches_tracked', 'features.1.0.block.1.fc1.weight', 'features.1.0.block.1.fc1.bias', 'features.1.0.block.1.fc2.weight', 'features.1.0.block.1.fc2.bias', 'features.1.0.block.2.0.weight', 'features.1.0.block.2.1.weight', 'features.1.0.block.2.1.bias', 'features.1.0.block.2.1.running_mean', 'features.1.0.block.2.1.running_var', 'features.1.0.block.2.1.num_batches_tracked', 'features.2.0.block.0.0.weight', 'features.2.0.block.0.1.weight', 'features.2.0.block.0.1.bias', 'features.2.0.block.0.1.running_mean', 'features.2.0.block.0.1.running_var', 'features.2.0.block.0.1.num_batches_tracked', 'features.2.0.b

/var/folders/jc/gg0g8k9x0n1cj1b8d04f93p80000gn/T/ipykernel_12404/4133761783.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(eye_model_path, map_l